In [ ]:
import pandas as pd

In [ ]:
#Defining the file to load
path="../Resources/purchase_data.csv"

#opening the data and displaying first 5 rows of the table
purchase_data_df=pd.read_csv(path)
purchase_data_df.head()

In [ ]:
#finding the number of unique SN to get the number of players
players=len(purchase_data_df["SN"].unique())
#displaying total number of players as data frame
total_players_df=pd.DataFrame({"Total Players":[players]})
total_players_df

In [ ]:
#running basic calculations 
items=purchase_data_df["Item Name"].nunique()
average_price=purchase_data_df["Price"].mean()
purchases_number = len(purchase_data_df["Purchase ID"])
revenue=purchase_data_df["Price"].sum()
#creating a summary table to hold the results
purchase_analysis_df=pd.DataFrame({"Number of unique items":[items],"Average Price":[average_price],"Number of Purchases":[purchases_number],
                    "Total Revenue":[revenue]})
#formatting the data 
purchase_analysis_df["Average Price"]=purchase_analysis_df["Average Price"].map("${:.2f}".format)
purchase_analysis_df["Total Revenue"]=purchase_analysis_df["Total Revenue"].map("${:,}".format)
purchase_analysis_df

In [ ]:
#grouping data frame by gender and counting unique values
gender=purchase_data_df.groupby('Gender').nunique()
#only selecting SN column to get count of male/female players
gender_count=gender["SN"]
#calculating the percentage
percentage=[]
index = [0,1,2]
for x in index:
    gender_pct=(gender_count[x]/players)*100
    percentage.append(gender_pct)
#creating new data frame to display demographics and formatting data
demographics_df=pd.DataFrame({"Total Count":gender_count,
                              "Percentage of Players":percentage})
demographics_df["Percentage of Players"]=demographics_df["Percentage of Players"].map("{:.2f}%".format)
demographics_df

In [ ]:
#calculating the purchase count and total purchase value by using groupby
purchase_count =purchase_data_df.groupby('Gender')["Purchase ID"].nunique()
total_purchase=purchase_data_df.groupby('Gender')["Price"].sum()
#calculating average purchase price for each gender
avg_price_gender=[]
index = [0,1,2]
for x in index:
    price_gender=(total_purchase[x]/purchase_count[x])
    avg_price_gender.append(price_gender)
#calculating average purchase price per person
price_person=[]
for n in index:
    price_person_gender=(purchase_count[n]/gender_count[n])*(avg_price_gender[n])
    price_person.append(price_person_gender)

In [ ]:
#creating data frame to display purchase analysis
purchase_by_gender_df=pd.DataFrame({"Purchase Count":purchase_count, 
                                    "Average Purchase Price":avg_price_gender,
                                    "Total Purchase Value":total_purchase,
                                   "Average Total Purchase per Person":price_person})
purchase_by_gender_df["Total Purchase Value"]=purchase_by_gender_df["Total Purchase Value"].map("${:,.2f}".format)
purchase_by_gender_df["Average Purchase Price"]=purchase_by_gender_df["Average Purchase Price"].map("${:.2f}".format)
purchase_by_gender_df["Average Total Purchase per Person"]=purchase_by_gender_df["Average Total Purchase per Person"].map("${:.2f}".format)
purchase_by_gender_df

In [ ]:
#Creating the bins in which age values will be placed
bins = [0, 9, 14, 19, 24,29,34,39,50]
#creating labels for the bins
group_labels = ["<10", "10-14", "15-19", "20-24","25-29","30-34","35-39","40+"]
#slicing the data and placing it in the bins
pd.cut(purchase_data_df["Age"],bins, labels=group_labels).head()

In [ ]:
#Adding data series into new column into the main DataFrame
purchase_data_df["Age Range"]=pd.cut(purchase_data_df["Age"],bins, labels=group_labels)
#creating groupby object based one the age range
age_data=purchase_data_df.groupby("Age Range")
#finding number of players in each age range
age_count =age_data["SN"].nunique()
age_count
percentage=[]
#looping through age column and calculating percentage
for number in age_count:
    age_percentage=(number/players)*100
    percentage.append(age_percentage)

In [ ]:
#creating data frame to display the results
#mapping the data to round the percentage column
age_demographics_df=pd.DataFrame({"Total Count":age_count, "Percentage of Players": percentage})
age_demographics_df["Percentage of Players"]=age_demographics_df["Percentage of Players"].map("{:.2f}%".format)
age_demographics_df

In [ ]:
#Utilizing age_data which already binned purchase data
#Calulcating purchase count, avg price, avg purchase per person and total purchase value
purchase_count=age_data["Purchase ID"].count()
avg_price_age=age_data["Price"].mean()
total_value_age=age_data["Price"].sum()
avg_per_person_age=[]
indexes=[0,1,2,3,4,5,6,7]
for x in indexes:
    per_person=(total_value_age[x]/age_count[x])
    avg_per_person_age.append(per_person)

In [ ]:
#Creating new dataframe to display above calculations
analysis_age = pd.DataFrame({"Purchase Count":purchase_count, "Average Purchase Price":avg_price_age,
                             "Total Purchase Value":total_value_age,"Avg Total Purchase per Person": avg_per_person_age})
analysis_age["Average Purchase Price"]=analysis_age["Average Purchase Price"].map("${:.2f}".format)
analysis_age["Total Purchase Value"]=analysis_age["Total Purchase Value"].map("${:.2f}".format)
analysis_age["Avg Total Purchase per Person"]=analysis_age["Avg Total Purchase per Person"].map("${:.2f}".format)
analysis_age

In [ ]:
#grouping data by SN to find 5 players who spent the most
top_purchase=purchase_data_df.groupby("SN")["Price"].sum().nlargest(5)
spenders=list(top_purchase.index)
#slicing data to display values only for top spenders
spenders_data=purchase_data_df.loc[(purchase_data_df["SN"]==spenders[0])| (purchase_data_df["SN"]==spenders[1])|
                            (purchase_data_df["SN"]==spenders[2])| (purchase_data_df["SN"]==spenders[3])|
                            (purchase_data_df["SN"]==spenders[4]),:]
#calculating purchase count
purchase_count=spenders_data.groupby("SN")["Purchase ID"].count()
spenders_summary = pd.DataFrame({"Purchase Count":purchase_count, "Total Purchase Value":top_purchase})
#looping through rows to calculate average purchase price
for rows in spenders_summary:
    avg_price_spenders=(spenders_summary["Total Purchase Value"]/spenders_summary["Purchase Count"])

In [ ]:
#creating and mapping data frame to display top spenders analysis
spenders_summary["Average Purchase Price"]=avg_price_spenders
spenders_summary["Average Purchase Price"]=spenders_summary["Average Purchase Price"].map("${:.2f}".format)
spenders_summary["Total Purchase Value"]=spenders_summary["Total Purchase Value"].map("${:.2f}".format)
spenders_summary=spenders_summary.sort_values(by=["Total Purchase Value"],ascending=False)
spenders_summary=spenders_summary[["Purchase Count","Average Purchase Price","Total Purchase Value"]]
spenders_summary

In [ ]:
#grouping data by item id to find 5 most popular items
top_item=purchase_data_df.groupby(["Item ID"])["Purchase ID"].count().nlargest(5)
items_index=list(top_item.index)
#slicing data to display data only for top items
items_data=purchase_data_df.loc[(purchase_data_df["Item ID"]==items_index[0])|(purchase_data_df["Item ID"]==items_index[1])|
                                (purchase_data_df["Item ID"]==items_index[2])|(purchase_data_df["Item ID"]==items_index[3])|
                                (purchase_data_df["Item ID"]==items_index[4]),:]
#grouping data by item id and item name to get multindex and performing calculations
best_items=items_data.groupby(["Item ID","Item Name"])["Purchase ID"].count()
purchase_value=items_data.groupby(["Item ID","Item Name"])["Price"].sum()
popular_items=pd.DataFrame({"Total Purchase Value":purchase_value,
                           "Purchase Count":best_items})
for rows in popular_items:
    item_price=(popular_items["Total Purchase Value"]/popular_items["Purchase Count"])

In [ ]:
#creating and mapping data frame to display top spenders analysis
popular_items["Item Price"]= item_price
popular_items["Item Price"]=popular_items["Item Price"].map("${:.2f}".format)
popular_items["Total Purchase Value"]=popular_items["Total Purchase Value"].map("${:.2f}".format)
popular_items=popular_items[["Purchase Count","Item Price","Total Purchase Value"]]
popular_items=popular_items.sort_values(by=["Purchase Count"],ascending=False)
popular_items

In [ ]:
#grouping data by item id to find 5 most profitable items
highest_price=purchase_data_df.groupby(["Item ID"])["Price"].sum().nlargest(5)
profitable_items_index=list(highest_price.index)
#slicing data to display data only for top items
top_items_data=purchase_data_df.loc[(purchase_data_df["Item ID"]==profitable_items_index[0])|
                                    (purchase_data_df["Item ID"]==profitable_items_index[1])|
                                    (purchase_data_df["Item ID"]==profitable_items_index[2])|
                                    (purchase_data_df["Item ID"]==profitable_items_index[3])|
                                    (purchase_data_df["Item ID"]==profitable_items_index[4]),:]
#grouping data by item id and item name to get multindex and performing calculations
items_count=top_items_data.groupby(["Item ID","Item Name"])["Purchase ID"].count()
total_values=top_items_data.groupby(["Item ID","Item Name"])["Price"].sum()
profitable_items_data=pd.DataFrame({"Total Purchase Value":total_values,
                           "Purchase Count":items_count})
for rows in profitable_items_data:
    item_price=(profitable_items_data["Total Purchase Value"]/profitable_items_data["Purchase Count"])

In [ ]:
#creating and mapping data frame to display most profitable items analysis
profitable_items_data["Item Price"]= item_price
profitable_items_data["Item Price"]=profitable_items_data["Item Price"].map("${:.2f}".format)
profitable_items_data["Total Purchase Value"]=profitable_items_data["Total Purchase Value"].map("${:.2f}".format)
profitable_items_data=profitable_items_data[["Purchase Count","Item Price","Total Purchase Value"]]
profitable_items_data=profitable_items_data.sort_values(by=["Total Purchase Value"],ascending=False)
profitable_items_data